In [ ]:

votes

,id,name,date,xls_url,pdf_url
0,20250318_3,"Gesetzentwurf zur Änderung der Artikel 109, 115 und 143h des Grundgesetzes",2025-03-18,https://www.bundestag.de/resource/blob/1057530/cd876d6eafbef0e924ab80fb63fe993a/20250318_3_xls.xlsx,https://www.bundestag.de/resource/blob/1057522/3108b8ae92d6ef722d4a8ca803af0321/20250318_3.pdf
1,20250318_2,"Antrag der FDP zur Änderung des Grundgesetzes (Artikel 109, 115 und 143h)",2025-03-18,https://www.bundestag.de/resource/blob/1057528/6332aea4e5ce3d333fe1e8471649eb30/20250318_2_xls.xlsx,https://www.bundestag.de/resource/blob/1057524/38ea2481939e815b6357d66d24166767/20250318_2.pdf
2,20250318_1,Gesetzentwurf der FDP zur Änderung des Grundgesetzes (Artikel 87a),2025-03-18,https://www.bundestag.de/resource/blob/1057526/a1b44089d98b02a457df9f9a1adb05e9/20250318_1_xls.xlsx,https://www.bundestag.de/resource/blob/1042658/e5de1ba939ebafd89d1148cfc22c47d6/20250318_1.pdf
3,20250131_1,„Zustrombegrenzungsgesetz“ der Fraktion der CDU/CSU,2025-01-31,https://www.bundestag.de/resource/blob/1042616/1ee9f3548b65c038dfb2a2c55495cd72/20250131_1_xls.xlsx,https://www.bundestag.de/resource/blob/1042614/84f41fdc1725dbbbd0cf4e9e5e1ea407/20250131_1.pdf
4,20250130_4,Bundeswehreinsatz im Mittelmeer „Eunavfor Med Irini“,2025-01-30,https://www.bundestag.de/resource/blob/1042564/bf6d97c18bb3ec6893335777d3e62e9b/20250130_4_xls.xlsx,https://www.bundestag.de/resource/blob/1042560/be89996d0aa5f5199fc7ef311922104b/20250130_4.pdf
...,...,...,...,...,...
701,20121025_4,"17/11193, Änderungsantrag zum Jahressteuergesetz 2013",2012-10-25,https://www.bundestag.de/resource/blob/192686/45d7fec00760588e049760c34a07da7f/20121025_4_xls.xls,https://www.bundestag.de/resource/blob/192688/cb9879e2ebfe24b79e881be43f6ef4e4/20121025_4.pdf
702,20121025_3,"17/11172, Änderungsantrag zum Gesetzentwurf zur Stärkung der deutschen Finanzaufsicht",2012-10-25,https://www.bundestag.de/resource/blob/192678/9e983f761c07aee7f89f72fa87024906/20121025_3_xls.xls,https://www.bundestag.de/resource/blob/192680/cd990de3a3f4f88f7a88914d7aac885f/20121025_3.pdf
703,20121025_2,"17/10059 und 17/11093, Abkommen zwischen Deutschland und der Schweiz",2012-10-25,https://www.bundestag.de/resource/blob/192676/2ffed073c6ac4a2bd600b83523e8d5a1/20121025_2_xls.xls,https://www.bundestag.de/resource/blob/192672/23eb9db1391cbb6780f2928b5599c44c/20121025_2.pdf
704,20121025,"17/10773 und 17/11174, Änderungen im Bereich der geringfügigen Beschäftigung",2012-10-25,https://www.bundestag.de/resource/blob/192674/db858b70d2821545c109b8c5ae752722/20121025_xls.xls,https://www.bundestag.de/resource/blob/192670/718f257f64a38a95139352f853820739/20121025.pdf


In [4]:
votes["id"].value_counts()

20130228      5
20130418      5
20130301      3
20130516      3
20130613      2
             ..
20210114_1    1
20201217_1    1
20201216_1    1
20201211_1    1
20121018      1
Name: id, Length: 691, dtype: int64

In [ ]:
votes["type"] = None

votes.loc[votes["name"].str.contains("Änderungsantrag"), "type"] = "Änderungsantrag"


In [3]:
votes

,id,name,date,xls_url,pdf_url
0,66e2e55e-85a8-4b05-9182-d37e232e5e43,"Gesetzentwurf zur Änderung der Artikel 109, 115 und 143h des Grundgesetzes",2025-03-18,https://www.bundestag.de/resource/blob/1057530/cd876d6eafbef0e924ab80fb63fe993a/20250318_3_xls.xlsx,https://www.bundestag.de/resource/blob/1057522/3108b8ae92d6ef722d4a8ca803af0321/20250318_3.pdf
1,1982f8d6-27c9-4f6b-a26f-191a3078d1c1,"Antrag der FDP zur Änderung des Grundgesetzes (Artikel 109, 115 und 143h)",2025-03-18,https://www.bundestag.de/resource/blob/1057528/6332aea4e5ce3d333fe1e8471649eb30/20250318_2_xls.xlsx,https://www.bundestag.de/resource/blob/1057524/38ea2481939e815b6357d66d24166767/20250318_2.pdf
2,bbbe162d-bbaf-4a80-8688-58e4541e85c0,Gesetzentwurf der FDP zur Änderung des Grundgesetzes (Artikel 87a),2025-03-18,https://www.bundestag.de/resource/blob/1057526/a1b44089d98b02a457df9f9a1adb05e9/20250318_1_xls.xlsx,https://www.bundestag.de/resource/blob/1042658/e5de1ba939ebafd89d1148cfc22c47d6/20250318_1.pdf
3,f60cd685-0d42-4b41-93be-39eb5c809845,„Zustrombegrenzungsgesetz“ der Fraktion der CDU/CSU,2025-01-31,https://www.bundestag.de/resource/blob/1042616/1ee9f3548b65c038dfb2a2c55495cd72/20250131_1_xls.xlsx,https://www.bundestag.de/resource/blob/1042614/84f41fdc1725dbbbd0cf4e9e5e1ea407/20250131_1.pdf
4,6bff8d86-38e8-4bfe-80bd-8472b275a9f0,Bundeswehreinsatz im Mittelmeer „Eunavfor Med Irini“,2025-01-30,https://www.bundestag.de/resource/blob/1042564/bf6d97c18bb3ec6893335777d3e62e9b/20250130_4_xls.xlsx,https://www.bundestag.de/resource/blob/1042560/be89996d0aa5f5199fc7ef311922104b/20250130_4.pdf
...,...,...,...,...,...
701,40dcc86a-ec9e-4e25-940d-c5a576147fe1,"17/11193, Änderungsantrag zum Jahressteuergesetz 2013",2012-10-25,https://www.bundestag.de/resource/blob/192686/45d7fec00760588e049760c34a07da7f/20121025_4_xls.xls,https://www.bundestag.de/resource/blob/192688/cb9879e2ebfe24b79e881be43f6ef4e4/20121025_4.pdf
702,b089230c-cbab-4533-af1e-c5b78c8d7044,"17/11172, Änderungsantrag zum Gesetzentwurf zur Stärkung der deutschen Finanzaufsicht",2012-10-25,https://www.bundestag.de/resource/blob/192678/9e983f761c07aee7f89f72fa87024906/20121025_3_xls.xls,https://www.bundestag.de/resource/blob/192680/cd990de3a3f4f88f7a88914d7aac885f/20121025_3.pdf
703,98e5cd44-11eb-43ae-b9ce-7005dfafd625,"17/10059 und 17/11093, Abkommen zwischen Deutschland und der Schweiz",2012-10-25,https://www.bundestag.de/resource/blob/192676/2ffed073c6ac4a2bd600b83523e8d5a1/20121025_2_xls.xls,https://www.bundestag.de/resource/blob/192672/23eb9db1391cbb6780f2928b5599c44c/20121025_2.pdf
704,ea6b9a48-c715-4692-b353-d018212937d1,"17/10773 und 17/11174, Änderungen im Bereich der geringfügigen Beschäftigung",2012-10-25,https://www.bundestag.de/resource/blob/192674/db858b70d2821545c109b8c5ae752722/20121025_xls.xls,https://www.bundestag.de/resource/blob/192670/718f257f64a38a95139352f853820739/20121025.pdf


In [13]:
pattern = r"(\d{8})_(?:(\d+))?"
import re

text = "https://www.bundestag.de/resource/blob/192676/2ffed073c6ac4a2bd600b83523e8d5a1/20121025_xls.xls"

match = re.search(pattern, text)
abstimmung_id = "_".join(match.groups())


TypeError: sequence item 1: expected str instance, NoneType found

In [1]:
from src.gather_vote_results import gather_vote_results
from src.gather_vote_texts import gather_vote_texts

gather_vote_texts()

2025-05-09 13:07:57.051 | INFO     | src.gather_vote_texts:gather_vote_texts:135 - Gathering vote texts from Bundestag website...
2025-05-09 13:07:57.090 | INFO     | src.gather_vote_texts:download_vote_texts:96 - Downloading https://dserver.bundestag.de/btd/20/151/2015117.pdf to data/tmp/texts/20250318_3/20250318_3_0.pdf
2025-05-09 13:07:57.460 | INFO     | src.utils.download:download_file:24 - Downloaded https://dserver.bundestag.de/btd/20/151/2015117.pdf to data/tmp/texts/20250318_3/20250318_3_0.pdf
2025-05-09 13:07:57.479 | INFO     | src.gather_vote_texts:download_vote_texts:96 - Downloading https://dserver.bundestag.de/btd/20/150/2015096.pdf to data/tmp/texts/20250318_3/20250318_3_1.pdf
2025-05-09 13:07:57.843 | INFO     | src.utils.download:download_file:24 - Downloaded https://dserver.bundestag.de/btd/20/150/2015096.pdf to data/tmp/texts/20250318_3/20250318_3_1.pdf
2025-05-09 13:07:57.863 | INFO     | src.gather_vote_texts:download_vote_texts:96 - Downloading https://dserver.bu